In [2]:
from services.user_dao import password_match, get_user_id
from services.portfolio_dao import get_portfolios_by_user
from services.transaction_dao import get_transactions_for_portfolio

In [ ]:
admin = "admin"
password = "admin"
if password_match(admin, password):
    print("Login successful")
    user_id = get_user_id(admin)
    print(f"User ID: {user_id}")
else:
    print("Login failed")

Login successful
User ID: 1


In [4]:
portfolios = get_portfolios_by_user(user_id)

In [5]:
portfolio_transactions = {}

for portfolio in portfolios:
    portfolio_id = portfolio["id"]
    transactions = get_transactions_for_portfolio(portfolio_id)
    portfolio_transactions[portfolio_id] = transactions

In [6]:
# Create mapping from portfolio ID to name
portfolio_id_to_name = {p["id"]: p["name"] for p in portfolios}

# Flatten and transform transactions into frontend-friendly format
frontend_transactions = []

for pid, transactions in portfolio_transactions.items():
    portfolio_name = portfolio_id_to_name.get(pid, "Unknown Portfolio")
    for tx in transactions:
        date_str = tx["executed_at"].strftime("%Y-%m-%d %H:%M:%S")  # properly formatted datetime
        frontend_transactions.append({
            "id": tx["id"],
            "portfolio": portfolio_name,
            "ticker": tx["ticker"],
            "type": tx["type"].title(),
            "quantity": tx["quantity"],
            "price": tx["price"],
            "dateTime": date_str
        })

In [4]:
from services.position_dao import buy_stock, sell_stock

In [5]:
buy_stock(1, "APPL", 10, 100.00)

TypeError: unsupported operand type(s) for +: 'decimal.Decimal' and 'float'